In [1]:
FILENAME = '1'

import time
import pickle
import numpy as np

WheelRadius = 32.5
WheelGap = 140
MaxRpm = 48/13*60*0.3
MinRpm = 24/13*60*0.3
MaxAngVel = MaxRpm*2*np.pi/60

In [ ]:
# Send movment commands to car and store outputs of rplidar and save with corresponding times
import utils

CarController = utils.CarController(WheelRadius, WheelGap, MaxRpm, MinRpm)

waypoints = np.array([[0                          , 0, 0, 0],
                      [MaxAngVel*WheelRadius*3    , 0, 0, 3],
                      [MaxAngVel*WheelRadius*3*1.8, 0, 0, 6],
                      [MaxAngVel*WheelRadius*3*1.8, 0, 0, 7]])
MotorWaypoints = CarController.WaypointsToMotorWaypoints(waypoints)
CarController.MotorController.drive(MotorWaypoints)

t0 = time.time()
waypoints[:, -1] += t0


try:
    out = []
    while True:
        val = CarController.GetLidar()
        print(val)
        
        out.append(val)
        
        if time.time() > waypoints[-1][-1]: break

finally:
    with open(FILENAME + '.pkl', 'wb') as handle:
        pickle.dump([out, waypoints], handle)

In [ ]:
with open(FILENAME + '.pkl', 'rb') as handle:
    out, waypoints = pickle.load(handle)

print(out)
print()
print(waypoints)

In [ ]:
# Run the server side code
import utils

with open(FILENAME + '.pkl', 'rb') as handle:
    _, waypoints = pickle.load(handle)

if len(waypoints) == 0:
    raise SystemExit("Empty File")

TimeOffset = utils.get()[0]
waypoints = np.array([i[:-1]+[i[-1]+TimeOffset] for i in waypoints.tolist()])

CarController = utils.CarController(WheelRadius, WheelGap, MaxRpm, MinRpm)
CarController.waypoints = waypoints
CarController.MotorWaypoints = CarController.WaypointsToMotorWaypoints(waypoints)
print(CarController.MotorWaypoints)
while True:
    CarController.main(debug=False, DebugVideo=True)

In [5]:
# Convert image files from CarController.DebugVideo to a video
import glob
import cv2
import os

dir = 'debug'

# Get sorted list of images
img_files = sorted(glob.glob(os.path.join(dir, '*.png')), key=lambda x: int(os.path.basename(x).split('.')[0]))

# Determine video size from first image
img = cv2.imread(img_files[0])
height, width, _ = img.shape

# Create video writer
video_name = os.path.join(dir, 'debug_video.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(video_name, fourcc, 2.0, (width, height))  # 2 FPS = 0.5s delay

for img_file in img_files:
    img = cv2.imread(img_file)
    video.write(img)
    
video.release()